In [ ]:
from aiida.orm import QueryBuilder
from aiida import load_profile
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData
load_profile()
from aiida.plugins import CalculationFactory, DataFactory


In [ ]:
qb = QueryBuilder()
qb.append(Node)
print(qb.count())

In [ ]:
###To get the Nodes from each user (Who) and When did they create
import numpy as np
try:
    import seaborn as sns
except:
    !pip install seaborn
    import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
    
qb = QueryBuilder()
qb.append(User)
Node_user = {'Number of Nodes':[],'Name of users':[]}


### for 4 users, count their number of tasks####
for user, in qb.all():
    
    qb1 = QueryBuilder()
    qb1.append(User, filters={'email': {'like': user.get_short_name()}},tag='user')
    qb1.append(Node,with_user='user')
    Node_user['Number of Nodes'] = Node_user['Number of Nodes'] + [qb1.count()];
    Node_user['Name of users'] = Node_user['Name of users'] + [user.get_short_name()];
    print('There are {} Nodes from user {}'.format(qb1.count(),user.get_full_name()))
    
### store this data for visulization


Node_user = pd.DataFrame(data = Node_user)

################# when did they create #####

## 1. first create the namelist
qb = QueryBuilder()
qb.append(User,project=['id','email'])
namelist = qb.all()
print('\nUsers and their IDs are: \n',namelist)
# define a function to get the name from name-id
def Namelist(x):
    return namelist[x-1][1]

## 2. then collect the date of creation
Node_time = {}
qb =QueryBuilder()
qb.append(Node,project=['ctime','user_id','id'])
Data_date_user = np.array(qb.all())

for data in Data_date_user:
    #Data_date_user[index][0] = Data_date_user[index][0].date()
    #Data_date_user[index][1] = Namelist(Data_date_user[index][1])
    data[0] = pd.to_datetime(data[0])
    data[1] = Namelist(data[1])

Plotdata = pd.DataFrame(Data_date_user,columns=['Create_Date','Name_of_user','Task ID'])
sns.histplot(data=Plotdata, x='Create_Date',hue='Name_of_user',bins=20)
#pd.value_counts(Plotdata['Create_Date'])
#plt.show()
Plotdata

In [ ]:
fig1, ax1 = plt.subplots(figsize=(25, 25))
ax1.pie(Node_user['Number of Nodes'],  autopct='%1.1f%%',shadow=False, startangle=90)
ax1.legend(title = 'users',labels=Node_user['Name of users'])
ax1.set_title("The users of Nodes")
Node_user

In [ ]:
## The number of Nodes of each type
from aiida.orm import Node
from aiida.orm import *
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np

Node.node_type

qb = QueryBuilder()
qb.append(Node)
Mydict = qb.all()
Mydict = np.ravel(Mydict)

Newdict = {}
for dic in Mydict:
    Newdict[dic.node_type] = Newdict.get(dic.node_type,0) + 1

#Finaldict = {'Name_Node':[],'Num_Node':[]}  
#for dic in Newdict:
    #Finaldict["Name_Node"] = Finaldict['Name_Node'] + [dic]
    #Finaldict['Num_Node'] = Finaldict['Num_Node'] + [Newdict[dic]]
Finaldict = {}

Finaldict['Type_Nodes'] = Newdict.keys()
Finaldict['Number_Nodes'] = Newdict.values()

Plotdict = pd.DataFrame(Finaldict)
Plotdict.sort_values(by=['Type_Nodes'],inplace = True)
#sn.barplot(Plotdict['Name_Node'],Plotdict['Num_Node'])

fig1, ax1 = plt.subplots(figsize=(25, 25), subplot_kw=dict(aspect="equal"))
#### explode used to make some parts explode####
explode = [0,0,0,0,0,0,0,0.1,0.1,0.1]
ax1.pie(Plotdict['Number_Nodes'],explode = explode ,autopct='%1.1f%%',shadow=False, startangle=90)
ax1.legend(labels=Plotdict['Type_Nodes'],title = 'Types of Nodes')
ax1.set_title("The type of Nodes")
Plotdict

In [ ]:
##the number of each process
from aiida.orm import Node,CalcJobNode
from aiida.orm import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Mydict = {}
qb = QueryBuilder()
qb.append(Node,filters = {'node_type':{'like':'process%'}})

for process in qb.all():
    Mydict[process[0].node_type] = Mydict.get(process[0].node_type,0)+1

Finaldict = {}

Finaldict['Type_Nodes'] = Mydict.keys()
Finaldict['Number_Nodes'] = Mydict.values()

Plotdict = pd.DataFrame(Finaldict)    

fig1, ax1 = plt.subplots(figsize=(10, 10),)
#### explode used to make some parts explode####

ax1.pie(Plotdict['Number_Nodes'],labels = Plotdict['Type_Nodes'],autopct='%1.1f%%',shadow=False, startangle=90)
#ax1.legend(labels=Plotdict['Type_Nodes'],title = 'Types of Nodes')
ax1.set_title("The type of Nodes")
Plotdict

In [ ]:
### Processing of aiida.orm.dic, no idea of which datas are interesting for us
from aiida.orm import Dict

qb = QueryBuilder()
qb.append(Dict)
qb.count()
#print(qb.all()[0][0].get_dict())
#print(qb.all()[5][0].attributes)
for dict, in qb.all():
    try:
        print(dict.get_attribute('material'))
    except:
        continue
    


In [ ]:
###### those 2 functions are modified, so they have no bugs(maybe) for count elements.
###### seperate string and Number, output is a list including element name and number of the element

from itertools import groupby
import re

def seperate_String_Num(s):
    
    groups = []
    uniquekeys = []
    for k, g in groupby(s, lambda x:x.isdigit()):
        groups.append(list(g))
        uniquekeys.append(k)
    if(uniquekeys[-1] == False):
        groups.append(['1'])
        uniquekeys.append(True)
    for i in range(len(groups)): 
        g = ''.join(groups[i])
        #if(uniquekeys[i]):
        #    g = int(g)
        groups[i] = g    
    #print(groups)
    return groups

def COUNTELEMENT(s):

    newstr = re.findall('[A-Z][^A-Z]*', s)
    mystr = {}
    for s in newstr:
        group = seperate_String_Num(s)
        mystr[group[0]] = group[1]
    return mystr


qb = QueryBuilder()
qb.append(StructureData)
COUNTELEMENT(qb.all()[0][0].get_formula())

In [ ]:
############ the processing of structure data ##################
from aiida.orm import StructureData
from itertools import groupby
import pandas as pd
import re

def addzero(s):
    for i in range(2*len(s)):### 2*len to prevent inserting reach the end when not finishing
            if(s[i].isdigit()==False):
                s.insert(i+1,'1')
def findnonzero(s,i):
    if(s[i]==False):
        return findnonzero(s,i-1)
    else:
        return i




##### parse input as structuredata  
def Count_Elements(Struc):
    groups = seperate_String_Num(Struc.get_formula())
    lis = {}
    ### since formula looks like X5Y8, we add number to the element column
    for i in range(len(groups)):
        if(groups[i].isdigit()==0):
            lis[groups[i]]=int(groups[i+1])
        else: 
            continue
    return lis


StrucList = []
qb = QueryBuilder()
qb.append(StructureData)
qb.count()

for struc, in qb.all()[:2]:
    StrucList = StrucList+ [COUNTELEMENT(struc.get_formula())]


pd.DataFrame(StrucList).fillna(0)

###### next step 